# Chapter 09

1️⃣ GAN 기본 모델 구현하기

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)

# 하이퍼파라미터 설정
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28 * 28
n_noise = 128

### 신경망 모델 구성  ###

# 비지도 학습이므로 Y를 사용하지 않음
X = tf.placeholder(tf.float32, [None,n_input])
Z = tf.placeholder(tf.float32, [None,n_noise])   # 노이즈를 입력할 플레이스홀더 Z

# 생성자 신경망에서 사용할 변수 설정
G_W1 = tf.Variable(tf.random_normal([n_noise,n_hidden], stddev = 0.01))  # 첫 번째 가중치, 편향 -> 은닉층으로 출력하기 위한 변수들
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev = 0.01))  # 두 번째 가중치, 편향 -> 출력층에 사용할 변수들
G_b2 = tf.Variable(tf.zeros(n_input))

# 구분자 신경망에서 사용할 변수 설정
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev = 0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev = 0.01))
D_b2 = tf.Variable(tf.zeros([1]))

# 생성자 신경망 구현
# 생성자는 무작위로 생성한 노이즈를 받아 가중치와 편향을 반영하여 은닉층을 만들고 은닉층에서 실제 이미지와 같은 크기의 결과를 출력하는 구성
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z,G_W1) + G_b1)  
    output = tf.nn.sigmoid(tf.matmul(hidden,G_W2) + G_b2)
    
    return output

# 구분자 신경망 구현
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    
    return output

# 무작위한 노이즈 만들어 주는 함수 
def get_noise(batch_size, n_noise):
    return np.random.normal(size = (batch_size, n_noise))

G = generator(Z)   # 노이즈 Z 이용해 생성자로 가짜 이미지 생성
D_gene = discriminator(G)   # G가 생성한 가짜 이미지를 구분자에 넣어 판별
D_real = discriminator(X)   # 진짜 이미지를 구분자에 넣어 판별

loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1-D_gene))   # D_real -> 1에 가까워야 함 / D_gene -> 0에 가까워야 함
loss_G = tf.reduce_mean(tf.log(D_gene))

# 생성자와 구별자 각각 loss구할 때 해당 신경망에 사용되는 변수들만 최적화해야 함
D_val_list = [D_W1,D_b1,D_W2,D_b2]
G_val_list = [G_W1,G_b1,G_W2,G_b2]

train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list = D_val_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list = G_val_list)

### 신경망 모델 학습  ###

sess=tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D , loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict = {X:batch_xs, Z:noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict = {Z:noise})
        
    print('Epoch:', '%04d' % epoch, 'D loss: {:.4}'.format(loss_val_D), 'G loss: {:.4}'.format(loss_val_G))
    
    # 확인용 이미지 생성
    if epoch == 0 or (epoch+1) % 10 ==0:  # 학습이 잘 되는지는 0,9,19,29번째,, ~ 마다 생성기로 이미지를 생성하여 눈으로 직접 확인
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict = {Z:noise})   # 노이즈를 만들고 G에 넣어 결과값 만듦
        
        # 결과값들을 28 * 28 크기의 가짜 이미지로 만들어 samples폴더에 저장하도록함
        fig, ax = plt.subplots(1, sample_size, figsize = (sample_size,1))
        
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i],(28,28)))
            
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)),bbox_inches ='tight')
        plt.close(fig)
        
print('최적화 완료!')

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Epoch: 0000 D loss: -0.6594 G loss: -2.023
Epoch: 0001 D loss: -0.3097 G loss: -2.441
Epoch: 0002 D loss: -0.115 G loss: -3.244
Epoch: 0003 D loss: -0.7724 G loss: -1.324
Epoch: 0004 D loss: -0.2095 G loss: -2.277
Epoch: 0005 D loss: -0.2602 G loss: -2.342
Epoch: 0006 D loss: -0.137 G loss: -3.055
Epoch: 0007 D loss: -0.08937 G loss: -3.77
Epoch: 0008 D loss: -0.1996 G loss: -2.762
Epoch: 0009 D loss: -0.3722 G loss: -2.281
Epoch: 0010 D loss: -0.2851 G loss: -2.64
Epoch: 0011 D loss: -0.2772 G loss: -2.688
Epoch: 0012 D loss: -0.3727 G loss: -2.561
Epoch: 0013 D loss: -0.2267 G loss: -3.018
Epoch: 0014 D loss: -0.3645 G loss: -2.723
Epoch: 0015 D loss: -0.3474 G loss: -2.752
Epoch: 0016 D loss: -0.3949 G loss: -2.646
Epoch: 0017 D loss: -0.3025 G loss: -2.75
Epoch: 0018 D loss: -0.4109 

2️⃣ 원하는 숫자 생성하기

In [13]:
# 노이즈에 레이블 데이터를 힌트로 넣어주는 방법을 사용

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)

# 하이퍼파라미터 설정
total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28 * 28
n_noise = 128
n_class = 10

### 신경망 모델 구성  ###

X = tf.placeholder(tf.float32, [None,n_input])
Y = tf.placeholder(tf.float32, [None,n_class])   # 결과값 판정용 아니고 노이즈와 실제 이미지에 해당하는 숫자를 힌트로 넣어주는 용도
Z = tf.placeholder(tf.float32, [None,n_noise])   # 노이즈를 입력할 플레이스홀더 Z

# 생성자 신경망 구현
# 변수들을 선언하지 않고 tf.layers사용 -> 변수 선언하지 않고 tf.variable_scope이용해 스코프 지정 가능
def generator_2(noise,labels):
    with tf.variable_scope('generator_2'):
        inputs = tf.concat([noise,labels],1)  # tf.concat함수를 이용해 noise값에 labels정보를 간단하게 추가
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        output = tf.layers.dense(hidden, n_input, activation=tf.nn.sigmoid)
    
    return output

# 구분자 신경망 구현
# 주의! 진짜 이미지를 판별할 때와 가짜 이미지를 판별할 때 똑같은 변수를 사용해야 함. 그러기 위해 scope.reuse_variables함수를 이용해 이전에 사용한 변수를 재사용하도록 구현.
def discriminator_2(inputs,labels, reuse = None):
    with tf.variable_scope('discriminator_2') as scope:
        if reuse :
            scope.reuse_variables()
            
        inputs = tf.concat([inputs,labels],1)
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        output = tf.layers.dense(hidden,1, activation=None)

    return output

# 무작위한 노이즈 만들어 주는 함수 -> 노이즈를 균등분포로 생성
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1.,1.,size = [batch_size, n_noise])

G = generator_2(Z,Y)   # 노이즈 Z 이용해 생성자로 가짜 이미지 생성. 레이블 정보 추가하여 레이블 정보에 해당하는 이미지 생성하도록 유도
D_gene = discriminator_2(G,Y)   # G가 생성한 가짜 이미지를 구분자에 넣어 판별
D_real = discriminator_2(X,Y,True)   # 진짜 이미지를 구분자에 넣어 판별. 사용한 변수들을 재사용하도록 reuse옵션을 true로!

loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_real, labels = tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_gene, labels = tf.zeros_like(D_gene)))
loss_D = loss_D_real + loss_D_gene  # 이 값을 최소화하면 구분자를 학습시킬 수 있음

loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_gene, labels = tf.ones_like(D_gene)))
    
# tf.get_collection 함수 이용해 구별자와 생성자 scope에서 사용된 변수들을 가져온 뒤 이 변수들을 최적화에 사용할 손실함수와 최적화 함수에 넣음
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'discriminator_2')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'generator_2')
train_D = tf.train.AdamOptimizer().minimize(loss_D, var_list = vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G, var_list = vars_G)

### 신경망 모델 학습  ###

sess=tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D , loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict = {X:batch_xs, Y:batch_ys,Z:noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict = {Y:batch_ys, Z:noise})
        
    print('Epoch:', '%04d' % epoch, 'D loss: {:.4}'.format(loss_val_D), 'G loss: {:.4}'.format(loss_val_G))
    
    # 확인용 이미지 생성
    if epoch == 0 or (epoch+1) % 10 ==0:  # 학습이 잘 되는지는 0,9,19,29번째,, ~ 마다 생성기로 이미지를 생성하여 눈으로 직접 확인
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        # 플레이스홀더 Y의 입력값을 넣어준다는 것이 위 코드와 다른점
        samples = sess.run(G, feed_dict = {Y:mnist.test.labels[:sample_size], Z:noise})   # 노이즈를 만들고 G에 넣어 결과값 만듦
        
        # 결과값들을 28 * 28 크기의 가짜 이미지로 만들어 samples폴더에 저장하도록함
        fig, ax = plt.subplots(2, sample_size, figsize = (sample_size,2))
        
        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()
            
            ax[0][i].imshow(np.reshape(mnist.test.images[i],(28,28)))
            ax[0][i].imshow(np.reshape(samples[i],(28,28)))
            
        plt.savefig('samples2/{}.png'.format(str(epoch).zfill(3)),bbox_inches ='tight')
        plt.close(fig)
        
print('최적화 완료!')

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Epoch: 0000 D loss: 0.008796 G loss: 7.532
Epoch: 0001 D loss: 0.01799 G loss: 7.157
Epoch: 0002 D loss: 0.01457 G loss: 7.024
Epoch: 0003 D loss: 0.005089 G loss: 7.678
Epoch: 0004 D loss: 0.03772 G loss: 7.238
Epoch: 0005 D loss: 0.1079 G loss: 6.165
Epoch: 0006 D loss: 0.0443 G loss: 7.801
Epoch: 0007 D loss: 0.0501 G loss: 7.76
Epoch: 0008 D loss: 0.08447 G loss: 9.119
Epoch: 0009 D loss: 0.08855 G loss: 6.449
Epoch: 0010 D loss: 0.158 G loss: 5.661
Epoch: 0011 D loss: 0.4178 G loss: 4.368
Epoch: 0012 D loss: 0.4011 G loss: 3.826
Epoch: 0013 D loss: 0.2167 G loss: 5.695
Epoch: 0014 D loss: 0.3046 G loss: 4.714
Epoch: 0015 D loss: 0.5512 G loss: 3.296
Epoch: 0016 D loss: 0.5108 G loss: 3.772
Epoch: 0017 D loss: 0.5404 G loss: 3.464
Epoch: 0018 D loss: 0.5879 G loss: 2.976
Epoch: 0019 